# Collaborative Filtering



## Zadání problému

V této ukázce bychom chtěli doporučovat filmy jen na základě žánrů k těmto přiřazených. Cílem tedy je nalézt filmy, které jsou si co do žánrového (kategoriálního) přiřazení nejpodobnější.

## Načtení dat

První fází je načtení zdrojových dat, která budeme chtít zpracovávat. Vstupní data jsou ve formátu `json` coby seznam položek o následujících atributech:
- `_id` typu `str` (unikátní identifikátor záznamu filmu)
- `name` typu `str` (název filmu)
- `year` typu `int` (rok natočení filmu)
- `genres` typu `str` (seznam žánrů filmu oddělený znakem lomítka `/`)
- `csfdUrl` typu `str` (reference na ČSFD) 

In [1]:
import json
import random
from dataclasses import dataclass

In [2]:
def load_file(file_path: str):
    with open(file_path) as file:
        data = json.load(file)
        file.close()
        return data

movies_data = load_file("./data/movies.json")
ratings_data = load_file("./data/ratings.json")

In [3]:
movies_data[:3]

[{'_id': '64d2a7963b7afeb6a7ff6b70',
  'name': 'Vykoupení z věznice Shawshank',
  'year': 1994,
  'genres': 'Drama / Krimi / Vězení',
  'csfdUrl': '2294-vykoupeni-z-veznice-shawshank'},
 {'_id': '64d2a7f53b7afeb6a7ff6b77',
  'name': 'Nedotknutelní',
  'year': 2011,
  'genres': 'Komedie / Drama / Životopisný',
  'csfdUrl': '306731-nedotknutelni'},
 {'_id': '64d2b5d6aacb468a3ff1a9a8',
  'name': 'Kmotr',
  'year': 1972,
  'genres': 'Drama / Krimi',
  'csfdUrl': '1644-kmotr'}]

In [4]:
ratings_data[:3]

[{'_id': '650592e51cca00cecf6b25a8',
  'rating': 4,
  'movie': '64f32aae3f4d83c88a192374',
  'user': '64f50be367062b1d8990f945'},
 {'_id': '650592f11cca00cecf6b25af',
  'rating': 4,
  'movie': '64f270b0375ee58978e1d120',
  'user': '64f50be367062b1d8990f945'},
 {'_id': '650592fa1cca00cecf6b25b6',
  'rating': 2,
  'movie': '64f4ccd7a5e05287b7cde545',
  'user': '64f50be367062b1d8990f945'}]

Pro prvotní manipulaci s daty využijme definice třídy, která nám usnadní a zpřehlední naše úvodní transformace. Využijme k tomu třídního dekorátoru `dataclass` z balíčku `dataclasses`.

In [5]:
@dataclass
class Movie:
    movie_id: str
    name: str
    year: int
    genres: list[str]

    def is_of_genre(self, genre: str) -> bool:
        return genre in self.genres


movies = []

def process_genres(genres_str: str) -> list[str]:
    return [genre.strip() for genre in genres_str.split("/")]

for movie in movies_data:
    movies.append(
        Movie(
            movie_id=movie["_id"],
            name=movie["name"],
            year=movie["year"],
            genres=process_genres(movie["genres"])
        )
    )

for movie in movies[:3]:
    print(movie)

Movie(movie_id='64d2a7963b7afeb6a7ff6b70', name='Vykoupení z věznice Shawshank', year=1994, genres=['Drama', 'Krimi', 'Vězení'])
Movie(movie_id='64d2a7f53b7afeb6a7ff6b77', name='Nedotknutelní', year=2011, genres=['Komedie', 'Drama', 'Životopisný'])
Movie(movie_id='64d2b5d6aacb468a3ff1a9a8', name='Kmotr', year=1972, genres=['Drama', 'Krimi'])


In [6]:
@dataclass
class Rating:
    rating_id: str
    movie_id: str
    user_id: str
    rating: int

    def is_of_user(self, user_id: str) -> bool:
        return self.user_id == user_id

    def is_of_movie(self, movie_id: str) -> bool:
        return self.movie_id == movie_id

    def means_unseen(self) -> bool:
        """Jestli hodnocení znamená, že uživatel film neviděl."""
        return self.rating == -1


ratings = []

for rating in ratings_data:
    ratings.append(
        Rating(
            rating_id=rating["_id"],
            movie_id=rating["movie"],
            user_id=rating["user"],
            rating=rating["rating"]
        )
    )

for rating in ratings[:3]:
    print(rating)

Rating(rating_id='650592e51cca00cecf6b25a8', movie_id='64f32aae3f4d83c88a192374', user_id='64f50be367062b1d8990f945', rating=4)
Rating(rating_id='650592f11cca00cecf6b25af', movie_id='64f270b0375ee58978e1d120', user_id='64f50be367062b1d8990f945', rating=4)
Rating(rating_id='650592fa1cca00cecf6b25b6', movie_id='64f4ccd7a5e05287b7cde545', user_id='64f50be367062b1d8990f945', rating=2)


Vždy se hodí mít i možnost rychlého přístupu k těmto datům pomocí filtrování a hledání. V praxi se k tomu obvykle využívá optimalizovaných knihoven jako je `numpy` a `pandas`; my takto hluboko zatím nepůjdeme.

In [7]:
def ratings_of_user(user_id: str, ratings_collection: list[Rating]) -> list[Rating]:
    """Přístupová funkce, která se pokusí vyfiltrovat hodnocení daného uživatele"""
    return [rating for rating in ratings_collection if rating.user_id == user_id]


def ratings_of_movie(movie_id: str, ratings_collection: list[Rating]) -> list[Rating]:
    """Přístupová metoda, která se pokusí vyfiltrovat hodnocení daného filmu."""
    return [rating for rating in ratings_collection if rating.movie_id == movie_id]


def user_rating_of_movie(movie_id: str, user_id: str, ratings_collection: list[Rating]) -> Rating:
    """Pokusí se dohledat hodnocení daného filmu od daného uživatele."""
    for rating in ratings_collection:
        if rating.movie_id == movie_id and rating.user_id == user_id:
            return rating

## Čištění dat

Před tím, než začneme skutečně něco s daty dělat, uvědomme si, že data nemáme kompletní. U tohoto typu dat nelze garantovat, že každý uživatel ohodnotil všechny filmy a (s tím související) že každý film byl dostatečně hodnocen.

V našem pohledu je také dobré zdůraznit, že neviděl-li uživatel film, je to pro nás (alespoň prozatím) známka toho, že _"film nehodnotil"_ - aktuálně pro nás tato skutečnost nepřináší přílišnou hodnotu. V budoucnu je to ale silný indikátor, že právě takový film bychom mohli doporučit primárně.

K tomu, jak se zbavit prázdných hodnot vede vícero cest. Uveďme si příklady:
- Ostranění záznamů - záznam, ve kterém něco chybí smažeme
- Doplnění náhodné hodnoty - záznam, ve kterém něco chybí, se pokusíme doplnit náhodnými hodnotami z povoleného rozsahu
- Doplnění průměrné hodnoty - chybějící hodnoty se pokusíme doplnit průměrnými hodnotami z daného sloupečku
- Doplnění nejčastější hodnoty - chybějící hodnoty doplníme nejčastější hodnotou z daného sloupečku
- Vážená průměrná hodnota - chybějící hodnotu doplníme průměrnou hodnotou váženou ke sledované skupině


### Jaká data nám chybí?

Máme zde problém, že někteří uživatelé neohodnotili vše. Dále pak že ne každý uživatel viděl všechny filmy a ohodnotil tyto pak s hodnotou `-1`. Další (a celkem zřejmý fakt) je, že některé filmy mají hodnocení pouze od malé skupiny uživatelů. Úkoly, které tedy doporučuji udělat:
- Odstranit uživatele, kteří mají méně než 20 % platných hodnocení (polovinu filmů neviděli nebo se neobtěžovali ohodnotit). Takový uživatel nám bez dalšího většího přidaného úsilí příliš nepomůže. S tím souvisí samozřejmě odstranit z dat i k nim přiřazená existující hodnocení, aby nám pak nedělaly problémy (konzistence).
- Odstranit filmy, které mají méně než 20 % platných hodnocení a pochopitelně opět k nim vytvořená existující hodnocení (přijdeme o velké množství informace ale o to budeme přesnější).
- Dopočítat chybějící hodnocení
   - Neviděl film (`-1`)
   - Neohodnotil film (neexistuje záznam v databázi - neobtěžoval se)


Všimněme si, že tento krok jsme při doporučování na základě obsahu vůbec dělat nemuseli - předpokládá se, že data uvedená coby atributy alternativ jsou již kompletní. Práce s uživatelskými daty jsou však vždy třeba zpracovat podstatně důsledněji.

---

Zde si předpočítáme pomocné proměnné

In [8]:
# Získejme unikátní uživatelská ID
user_ids = list(set([rating.user_id for rating in ratings]))

# Počet filmů
n_movies = len(movies)

Nyní si stanovme kvóty pro filtrování

In [9]:
# Minimální kvóta pro platná uživatelských hodnocení
user_ratings_requirement = 0.2

# Minimální kvóta pro platná hodnocení filmů
movie_ratings_requirement = 0.2

In [10]:
users_to_be_removed = []

for user_id in user_ids:
    user_ratings = ratings_of_user(user_id, ratings)
    valid_user_ratings = [r for r in user_ratings if r.rating > 0]
    
    if len(valid_user_ratings) < n_movies * user_ratings_requirement:
        users_to_be_removed.append(user_id)
        print(user_id)


for removable_user in users_to_be_removed:
    # Vyfiltruj seznamy tak, aby nebyl záznam o daném uživateli
    user_ids = [user_id for user_id in user_ids if user_id != removable_user]
    ratings = [rating for rating in ratings if rating.user_id != removable_user]

65071c950358f547b0248bac
65229906f81f149a378012c7


In [11]:
movies_to_be_removed = []

for movie in movies:
    movie_ratings = ratings_of_movie(movie.movie_id, ratings)
    valid_movie_ratings = [r for r in movie_ratings if r.rating > 0]

    if len(valid_movie_ratings) < len(user_ids) * movie_ratings_requirement:
        movies_to_be_removed.append(movie.movie_id)
        print(movie)


for removable_movie in movies_to_be_removed:
    movies = [m for m in movies if m.movie_id != removable_movie]
    ratings = [r for r in ratings if r.movie_id != removable_movie]

Movie(movie_id='64f273f8375ee58978e1d15c', name='Město bohů', year=2002, genres=['Krimi', 'Drama'])
Movie(movie_id='64f27853375ee58978e1d224', name='Čas probuzení', year=1990, genres=['Drama', 'Životopisný'])
Movie(movie_id='64f27867375ee58978e1d228', name='25 let neviny', year=2020, genres=['Drama', 'Vězení'])
Movie(movie_id='64f4d03aa5e05287b7cde589', name='Skrytá čísla', year=2016, genres=['Životopisný', 'Historický', 'Drama', 'Věda'])


### Doplnění chybějících dat

Pro ilustraci bychom mohli využít možnosti dopočítat chybějící hodnocení trochu komplexněji. Abychom zachovali jak informaci o hodnocení uživatele (zda dává převážně negativní či pozitivní hodnocení), tak i o tom, jak je film hodnocen průměrně, můžeme zařadit všechna hodnocení daného uživatele a hodnocení daného filmu do jednoho dlouhého seznamu. Z toho již spočítáme aritmetický průměr snadno, jak jsme zvyklí. 

To provedeme pro všechna chybející hodnocení (tedy neexistuje záznam v databázi), tak pro hodnocení o tom, že uživatel film neviděl.

In [12]:
def calc_average_rating(ratings_to_calc: list[Rating]) -> float:
    """Vypočítá průměrné hodnocení z dodané skupiny hodnocení"""
    return sum(r.rating for r in ratings_to_calc) / len(ratings_to_calc)

for user_id in user_ids:
    user_ratings = ratings_of_user(user_id, ratings)
    seen_ratings = [r for r in user_ratings if not r.means_unseen()]
    not_seen_ratings = [r for r in user_ratings if r.means_unseen()]

    for unseen in not_seen_ratings:
        movie_ratings = [r for r in ratings_of_movie(unseen.movie_id, ratings) if not r.means_unseen()]
        unseen.rating = calc_average_rating(seen_ratings + movie_ratings)

In [13]:
for idx, user_id in enumerate(user_ids):
    for movie in movies:
        user_rating = user_rating_of_movie(movie.movie_id, user_id, ratings)
        if not user_rating:
            user_ratings = ratings_of_user(user_id, ratings)
            movie_ratings = ratings_of_movie(movie.movie_id, ratings)
            ratings.append(Rating(
                rating_id=f"generated-{idx}",
                movie_id=movie.movie_id,
                user_id=user_id,
                rating=calc_average_rating(user_ratings + movie_ratings)
            ))

Nyní si ověřme, že každý uživatel má přiřazeno hodnocení k nějakému filmu (byť vygenerované).

In [14]:
for user_id in user_ids:
    user_ratings = ratings_of_user(user_id, ratings)
    if len(user_ratings) != len(movies):
        print(user_id, len(user_ratings))

### Mechanismy výpočtu podobnosti

Stejně jako v příkladě na filtrování na základě obsahu se podívejme na metriky podobnosti, které budeme používat. Kokrétně tedy na vzdálenost ***euklidovskou*** a ***kosínovou***.

Euklidovská vzdálenost je základní metrikou, kterou můžeme použít pro počítání rozdílů mezi dvěma body v `n`-dimenzionálním prostoru. Využijeme následující vzorec, který lze snadno odvodit z pythagorovy věty pro použití v `n`-dimenzionálním prostoru:

$$d_{euklid} = \sqrt{(x_1 - y_1)^2 + (x_2 - y_2)^2 + \ldots + (x_n - y_n)^2}$$


Pomocí tohoto vzorce jsme schopni spočítat, že některé filmy jsou si podobné na základě žánrů.

In [15]:
def euclidean_distance(point1: tuple[float], point2: tuple[float]) -> float:
    """Spočítá euklidovskou vzdálenost mezi dvěma mnohadimenzionálními body.
    Pokud body nejsou stejné dimenzionality, vyhazuje chybu.
    """
    if len(point1) != len(point2):
        raise ValueError(f"Body musí být stejné dimenzionality: {len(point1)} != {len(point2)}")

    differences = []
    # Pro každou dimenzi
    for idx, value_in_dim in enumerate(point1):
        # Přidej absolutní rozdíl mezi hodnotami v dimenzi
        differences.append(abs(value_in_dim - point2[idx]))

    # Převeď absolutní rozdíly na své druhé mocniny
    squared_differences = [diff ** 2 for diff in differences]

    # Odmocni umocněné rozdíly
    return sum(squared_differences) ** 0.5

Kosínová vzdálenost je druhou významnou metrikou, která nám dokáže spočítátat podobnost dvou bodů v `n`-dimenzionálním prostoru. Pracuje s úhlem, který dva dané vektory vzájemně svírají.

Vzorec pro kosínovu větu říká, že je-li úhel svíraný mezi danými dvěma vektory nulový, návratová hodnota bude `1`. Samotný vzorec pak vypadá následovně:

$$cos(\phi) = \frac{\vec{u} \cdot \vec{v}}{|\vec{u}| \cdot |\vec{v}|}$$


My z tohoto pravidla vyjdeme a trochu si vzoreček pro potřeby výpočtu této vzdálenosti (_i.e._ podobnosti) upravíme:

$$d_{cosine} = 1 - \frac{\vec{u} \cdot \vec{v}}{|\vec{u}| \cdot |\vec{v}|}$$


Tím zajistíme, že čím menší velikost svíraného úhlu, tím více se výsledná hodnota blíží nule. Zároveň zde máme z povahy řešeného problému garantováno, že nedojde k extrémním situacím:
- pracujeme v I. kvadrantu (svíraný úhel nemůže být větší než 90°)
- neexistuje film bez přiřazeného žánru (neexistuje tedy nulový vektor a tedy nenastane situace, kdy bychom dělili nulou)

Dané body můžeme chápat jako vektory od počátku k danému bodu. Tím získáme výhodu, že jsou-li vektory lineárně závislé (ve zkratce že leží na stejné přímce, resp. $k \cdot \vec{u} = \vec{v}$), bude vzdálenost mezi nimi chápána jako 0 (tedy totožné). Samozřejmě zde můžeme zobecnit tezi na *"čím menší úhel dva libovolně velké vektory svírají, tím jsou si podobnější"*.

In [16]:
def cosine_distance(u: tuple[float], v: tuple[float]) -> float:
    """Spočítá kosínovou vzdálenost mezi dvěma mnohadimenzionálními vektory.
    Pokud vektory nejsou stejné dimenzionality, vyhazuje chybu. Stejně tak pokud
    je některý z vektorů tzv. nulový (o nulové délce) - dělili bychom nulou.
    """
    if len(u) != len(v):
        raise ValueError(f"Vektory musí být stejné dimenzionality: {len(u)} != {len(v)}")

    # Nejdříve si spočteme čitatele - součin dvou vektorů (tzv. dot product)
    dot_product = sum(u_n * v_n for u_n, v_n in zip(u, v))

    # Připravíme si pomocnou funkci pro výpočet délky vektoru (pomocí pythagorovy věty)
    vector_lenght = lambda vector: sum(x ** 2 for x in vector) ** 0.5

    u_len = vector_lenght(u)
    v_len = vector_lenght(v)

    if u_len == 0 or v_len == 0:
        raise ValueError(f"Byl dodán nulový vektor")

    return 1 - (dot_product / (u_len * v_len))

## Strategie User-User Matrix

Tato metoda je postavena na hledání podobností mezi uživateli systému. Běžně se řeší pomocí matice, my si to celé trochu zjednodušíme. V tomto případě budeme hledat jen vzájemně nejbližší uživatele dle svých hodnocení.

Každého uživatele si opět převedeme do podoby, kdy pro každé hodnocení daného uživatele vybereme jen a pouze dané číslo coby hodnocení. Z této sekvence pak budeme hledat nejbližší sousedy. Budeme tedy mít pro každého uživatele n-tici obsahující ID uživatele (pro zpětnou referenci) a dlouhou posloupnost hodnocení (stabilně seřazenou dle filmů). Uvědomme si, že tímto nám vznikne `n`-rozměrný prostor, ve kterém se `n` rovná počtu filmů (v našem případě zhruba 300). To by pro větší datasety mohlo být již náročnější sousto.

In [17]:
def couple_items(items_to_be_coupled: list) -> list[tuple]:
    """Vytvoří jednosměrné dvojice z dodaného seznamu položek."""
    result_tuples = []
    
    for idx, u1 in enumerate(items_to_be_coupled[:-1]):
        for u2 in items_to_be_coupled[idx + 1:]:
            result_tuples.append((u1, u2))

    return result_tuples

In [18]:
users_as_points = []

for user_id in user_ids:

    # Seřazeny filmy podle ID filmu aby bylo garantováno pořadí
    user_movie_ratings = sorted(
        [rating for rating in ratings_of_user(user_id, ratings)],
        key=lambda rating: rating.movie_id
    )

    users_as_points.append((
        user_id,
        *[rating.rating for rating in user_movie_ratings]
    ))      

Nyní si spárujme naše uživatele reprezentovatelné jako body v n-rozměrném prostoru.

In [19]:
user_couples = couple_items(users_as_points)

Zkusme to nejdříve s euklidovskou vzdáleností

In [20]:
user_couple_distances_euclidean = []

for user_couple in user_couples:
    user_couple_distances_euclidean.append((
        # Uložme si ID daných uživatelů
        user_couple[0][0],
        user_couple[1][0],

        # Spočítejme euklidovskou vzdálenost mezi body
        # reprezentujícími dané uživatele
        euclidean_distance(user_couple[0][1:], user_couple[1][1:])
    ))

In [21]:
# Vyber 20 nejbližších
sorted(user_couple_distances_euclidean, key=lambda couple: couple[2])[:20]

[('6514515a5920516a950ecdf9', '651451605920516a950ece17', 9.909193420997306),
 ('6514515c5920516a950ece08', '6514515a5920516a950ecdf9', 10.43915658621663),
 ('6514515c5920516a950ece08', '651451605920516a950ece17', 10.8752116333766),
 ('650b5b157c75dd6fd5b1babd', '6514515a5920516a950ecdf9', 11.365673842348022),
 ('6514515e5920516a950ece0e', '6514515a5920516a950ecdf9', 11.39846362366874),
 ('6514515e5920516a950ece0e', '651451605920516a950ece17', 11.638651620417862),
 ('6514515a5920516a950ecdf9', '6522a278f81f149a37801335', 11.79484336273568),
 ('651451605920516a950ece17', '651451595920516a950ecdf6', 11.928929922914657),
 ('6507260e2050e9e1f928cab5', '6514515a5920516a950ecdf9', 11.976427873556743),
 ('6522a53bf81f149a37801344', '651451625920516a950ece23', 12.045242937561595),
 ('651451615920516a950ece1a', '651451625920516a950ece23', 12.059403817989217),
 ('650b5b157c75dd6fd5b1babd', '6522a278f81f149a37801335', 12.113462696630688),
 ('6522a53bf81f149a37801344', '651451615920516a950ece1a', 

In [22]:
# Vyber 20 náhodných
random.sample(user_couple_distances_euclidean, 20)

[('65229b71f81f149a378012cc', '6506c5e1ecd687f1fffbd5b8', 23.44998759706008),
 ('6514515e5920516a950ece11', '6505bf0394002f289a99d34f', 20.80913171399081),
 ('650b3b90adb3b7afd6cddbfb', '650b2cfd7237ff293a039982', 17.475942232714548),
 ('65229cc5f81f149a378012e5', '651451595920516a950ecdf3', 25.475355042416716),
 ('6514515d5920516a950ece0b', '65229b71f81f149a378012cc', 17.76310775381422),
 ('6506c5e1ecd687f1fffbd5b8', '651451625920516a950ece20', 21.556163121457967),
 ('6514515a5920516a950ecdf9', '6506c5e1ecd687f1fffbd5b8', 21.826012767563352),
 ('6514515c5920516a950ece08', '6514515b5920516a950ecdff', 20.47314177780506),
 ('65229cc5f81f149a378012e5', '6505bf4994002f289a99d362', 16.936746502239714),
 ('651451595920516a950ecdf3', '651451625920516a950ece23', 24.94469647445887),
 ('6505cf9694002f289a99d3b0', '6505bf4994002f289a99d362', 21.914738376469273),
 ('6522a3a2f81f149a3780133a', '6514515b5920516a950ecdff', 19.302207104534723),
 ('6522a278f81f149a37801335', '651451595920516a950ecdf6',

### Jaké je praktické použití?

V praxi si lze v zásadě představit několik strategií (které pochopitelně ovlivňují i vypočítávaný výstup):
- **Pravidelné napočítávání** - např. každou noc se napočítávají takovéto výsledky; každému uživateli se pak doporučují produkty podle nejpodobnějších (nejbližších) uživatelů
- **Real-time počítání** - při každém dotazu se vypočítá aktuální nejbližší uživatel (z tohoto pohledu bychom však asi i tento dataset chápali jako velmi obtížný)
- **Stanovení refernčních archetypů uživatelů** - pravidelně se stanovují typičtí uživatelé, kteří reprezentují nějaký uživatelský segment (více najdete pod pojmem *cluster analysis* - povíme si později). Uživatelé se mají tendenci sdružovat do shluků a tedy této vlastnosti lze využívat. Pak by v oblasti naší úlohy mohlo stačit identifikovat těžiště takových shluků uživatelů a přiřadit Vás k němu. Mohli bychom si to představit třeba nějak takhle:
   - Macho divák (převažují akční filmy, rychlá auta a thrillery, nikdy nedávají dobré hodnocení muzikálům)
   - Geek (milují sci-fi a fantasy, nepotrpí si na horory)
   - Romantik (muzikály, dobrodružné a především romantické filmy; nesnáší horory)
   - Veselá kopa (absurdní komedie miluje, zvládne i komediální akční či animované filmy)
   - ...


Nyní si zkusme ilustrovat první případ - jste evidovaný uživatel a chcete najít nejbližší doporučení. 

In [23]:
def closest_users(user_id: str, n: int = 5) -> list[tuple[str, str, float]]:
    """Pokusí se vyhledat `n` nejpodobněji hodnotících uživatelů"""
    # Zkrácený zápis
    couples = user_couple_distances_euclidean
    filtered = [couple for couple in couples if couple[0] == user_id or couple[1] == user_id]
    return sorted(filtered, key=lambda couple: couple[2])[:n]

In [24]:
closest_users("6514515a5920516a950ecdfc", 10)

[('6514515a5920516a950ecdfc', '6522a53bf81f149a37801344', 30.437462577262647),
 ('6514515a5920516a950ecdfc', '651451625920516a950ece23', 30.539325983725288),
 ('6514515a5920516a950ecdfc', '650da95f16593c65ac62d605', 30.786728630497915),
 ('6514515a5920516a950ecdfc', '6514515f5920516a950ece14', 30.82562676869674),
 ('6514515a5920516a950ecdfc', '6514515b5920516a950ecdff', 31.191651631814025),
 ('6514515a5920516a950ecdfc', '6522a3fdf81f149a3780133f', 31.612578505465578),
 ('6522a3a2f81f149a3780133a', '6514515a5920516a950ecdfc', 32.12589735874451),
 ('6514515a5920516a950ecdfc', '650727672050e9e1f928cac8', 32.24616240387768),
 ('6514515a5920516a950ecdfc', '6514515e5920516a950ece11', 32.361126764565306),
 ('65229cc5f81f149a378012e5', '6514515a5920516a950ecdfc', 32.45192851445802)]

Nyní proveďme to samé se vzdáleností kosínovou

In [25]:
user_couple_distances_cosine = []

for user_couple in user_couples:
    user_couple_distances_cosine.append((
        # Uložme si ID daných uživatelů
        user_couple[0][0],
        user_couple[1][0],

        # Spočítejme kosínovou vzdálenost mezi vektory
        # reprezentujícími dané uživatele
        cosine_distance(user_couple[0][1:], user_couple[1][1:])
    ))

In [26]:
# Vyber 10 nejbližších
sorted(user_couple_distances_cosine, key=lambda couple: couple[2])[:10]

[('6514515a5920516a950ecdf9', '651451605920516a950ece17', 0.00818530334161982),
 ('6514515c5920516a950ece08',
  '6514515a5920516a950ecdf9',
  0.009503487445327363),
 ('6514515c5920516a950ece08',
  '651451605920516a950ece17',
  0.009724810960486052),
 ('6514515a5920516a950ecdf9',
  '651451625920516a950ece23',
  0.009739932313826194),
 ('6514515e5920516a950ece0e', '6514515a5920516a950ecdf9', 0.00981530194128899),
 ('650b5b157c75dd6fd5b1babd',
  '6514515a5920516a950ecdf9',
  0.010201662718212212),
 ('651451605920516a950ece17',
  '651451625920516a950ece23',
  0.010329423354077272),
 ('6514515a5920516a950ecdf9',
  '651451615920516a950ece1a',
  0.010634938914453551),
 ('6514515e5920516a950ece0e', '651451605920516a950ece17', 0.01072133414099885),
 ('6514515a5920516a950ecdf9',
  '6522a278f81f149a37801335',
  0.010956414806312265)]

In [27]:
# Vyber 10 náhodných
random.sample(user_couple_distances_cosine, 10)

[('6505cf2e94002f289a99d3ab', '64f50be367062b1d8990f945', 0.1402586851896287),
 ('6514515d5920516a950ece0b',
  '651451625920516a950ece23',
  0.014622378272956538),
 ('65229cc5f81f149a378012e5', '64f50be367062b1d8990f945', 0.07185943540364359),
 ('6505cf9694002f289a99d3b0',
  '651451595920516a950ecdf6',
  0.037691973822133185),
 ('650da95f16593c65ac62d605', '64f50be367062b1d8990f945', 0.08777393795139032),
 ('650da95f16593c65ac62d605',
  '6522a3fdf81f149a3780133f',
  0.037272484152745644),
 ('650b5b157c75dd6fd5b1babd', '6514515b5920516a950ece02', 0.05859647881518393),
 ('65229b71f81f149a378012cc', '6522a278f81f149a37801335', 0.02195110767657449),
 ('652ba0b2b1d18a296fc3eca6', '6514515c5920516a950ece05', 0.08338333275648979),
 ('6514515a5920516a950ecdf9',
  '6522a3fdf81f149a3780133f',
  0.023267667399718972)]

## Strategie Item-Item

In [28]:
movies_as_points = []

for movie in movies:
    movie_ratings = ratings_of_movie(movie.movie_id, ratings)
    sorted_data = sorted(movie_ratings, key=lambda rating: rating.user_id)
    point_data = [rating.rating for rating in sorted_data]
    movies_as_points.append((
        movie.name,
        *point_data
    ))

movies_as_points[0]

('Vykoupení z věznice Shawshank',
 2,
 5,
 5,
 5,
 3,
 3.9519309368261837,
 5,
 5,
 4.2549962990377495,
 3.7376710159283904,
 4,
 4,
 3,
 4,
 5,
 3.2141376241051614,
 4,
 4.275858485154953,
 5,
 4,
 2,
 5,
 5,
 5,
 5,
 3.6891191709844557,
 3.671875185664617,
 5,
 5,
 3.9087173543782066,
 3.8883484995700375,
 5,
 4,
 3.9955922219667612,
 5,
 5,
 3.6836992702831735,
 5,
 5)

In [29]:
coupled_movies = couple_items(movies_as_points)

In [30]:
coupled_movies_euclidean = []

for movie_couple in coupled_movies:
    coupled_movies_euclidean.append((
        movie_couple[0][0],
        movie_couple[1][0],

        euclidean_distance(movie_couple[0][1:], movie_couple[1][1:])
    ))

In [31]:
# Vyber 20 nejbližších
sorted(coupled_movies_euclidean, key=lambda couple: couple[2])[:20]

[('L. A. - Přísně tajné', 'Obvyklí podezřelí', 1.746603208396551),
 ('Vynález zkázy', 'Pro pár dolarů navíc', 2.165240779945492),
 ('Uprchlík', 'Medvěd', 2.22525541063766),
 ('Butch Cassidy a Sundance Kid', 'Poslední skaut', 2.2909686010587347),
 ('Podraz', 'Škola základ života', 2.3835907069936186),
 ('Četa', 'Medvěd', 2.4048219813221254),
 ('Butch Cassidy a Sundance Kid', 'Motýlek', 2.4487508455963396),
 ('Poslední skaut', 'Ponorka', 2.4788963523998784),
 ('Obvyklí podezřelí', 'Prezident Blaník', 2.5053849215316877),
 ('Hlídač č. 47', 'Letec', 2.553126781781749),
 ('Obvyklí podezřelí', 'Medvěd', 2.6307388770870865),
 ('Americký gangster', 'Tropická bouře', 2.6481962145625997),
 ('Prezident Blaník', 'Medvěd', 2.652710098650573),
 ('Motýlek', 'Medvěd', 2.72774219369514),
 ('Prezident Blaník', 'Atentát', 2.7502567281047026),
 ('Butch Cassidy a Sundance Kid', 'Ponorka', 2.7618799885994223),
 ('Motýlek', 'Gauneři', 2.764357597864925),
 ('Casino', 'Pro pár dolarů navíc', 2.787263775557484)

In [32]:
# Vyber 20 náhodných
random.sample(coupled_movies_euclidean, 20)

[('Nedotknutelní', 'Já, padouch', 8.218893590986738),
 ('Život je krásný', 'Přednosta stanice', 6.501246028201765),
 ('L. A. - Přísně tajné', 'Medvěd', 3.7248417924069366),
 ('Hlídač č. 47', 'Whiplash', 4.9573922300842455),
 ('Captain Marvel', 'Občanský průkaz', 7.786851154695899),
 ('Gran Torino', 'Rain Man', 5.5943333235206625),
 ('Temný rytíř', 'Mariňák', 6.693337371018181),
 ('U pokladny stál...', 'Rok jedna', 5.926303852955756),
 ('Čistá duše', 'U pokladny stál...', 6.390755269358084),
 ('Dvanáct rozhněvaných mužů', 'Vrchní, prchni', 6.348271538901555),
 ('Casino', 'Mr. & Mrs. Smith', 6.940375212890305),
 ('Coco', 'Star Wars: Epizoda VI - Návrat Jediů', 6.79646450837165),
 ('Coco', 'Rocky', 8.344578539979498),
 ('Počátek', 'Život Briana', 7.496171515090434),
 ('Transformers', 'Lara Croft - Tomb Raider', 7.693635349044115),
 ('12 opic', 'Atentát', 3.959368898742413),
 ('Barbie', 'Záhada Blair Witch', 7.256542051578029),
 ('Star Wars: Epizoda IV - Nová naděje', 'Whiplash', 8.3898568

In [33]:
coupled_movies_cosine = []

for movie_couple in coupled_movies:
    coupled_movies_cosine.append((
        movie_couple[0][0],
        movie_couple[1][0],

        cosine_distance(movie_couple[0][1:], movie_couple[1][1:])
    ))

In [34]:
# Vyber 20 nejbližších
sorted(coupled_movies_cosine, key=lambda couple: couple[2])[:20]

[('L. A. - Přísně tajné', 'Obvyklí podezřelí', 0.0026946970235649514),
 ('Vynález zkázy', 'Pro pár dolarů navíc', 0.004253525836561245),
 ('Uprchlík', 'Medvěd', 0.004477321183861793),
 ('Butch Cassidy a Sundance Kid', 'Poslední skaut', 0.004677895838930857),
 ('Podraz', 'Škola základ života', 0.004816624938865921),
 ('Butch Cassidy a Sundance Kid', 'Motýlek', 0.004909586683826461),
 ('Četa', 'Medvěd', 0.004951070581973527),
 ('Poslední skaut', 'Ponorka', 0.0049618447968151624),
 ('Hlídač č. 47', 'Letec', 0.005680581011252461),
 ('Obvyklí podezřelí', 'Prezident Blaník', 0.005775341424990588),
 ('Motýlek', 'Medvěd', 0.005791631146873422),
 ('Prezident Blaník', 'Atentát', 0.005921985042079192),
 ('Butch Cassidy a Sundance Kid', 'Ponorka', 0.006046581909566862),
 ('Americký gangster', 'Tropická bouře', 0.006224509823856916),
 ('Obvyklí podezřelí', 'Medvěd', 0.006376581777966739),
 ('Havel', 'Zátopek', 0.006540863570232203),
 ('Americká krása', 'Demolition Man', 0.006564268109616633),
 ('Pr

In [35]:
# Vyber 20 náhodných
random.sample(coupled_movies_cosine, 20)

[('Americká krása', 'Nekonečný příběh', 0.034676539504116866),
 ('U pokladny stál...', 'The Avengers', 0.036112332946976355),
 ('Transformers', 'Hanebný pancharti', 0.05852885105925809),
 ('Kmotr II', 'Podfu(c)k', 0.03490396368234849),
 ('Dnes neumírej', 'Kill Bill', 0.032526689008544496),
 ('Marečku, podejte mi pero!', 'Čistá duše', 0.03529483985054893),
 ('Titanic', 'Muž na Měsíci', 0.06480509397310263),
 ('Spider-Man: Napříč paralelními světy',
  'Na samotě u lesa',
  0.060028103450549763),
 ('Pařba v Bangkoku', 'Zátopek', 0.05691848759016016),
 ('Příšerky s.r.o.', 'Teorie všeho', 0.051823521285630725),
 ('12 opic', 'Captain Marvel', 0.03782542763255958),
 ('Králova řeč', 'Neuvěřitelný Hulk', 0.0499476090954577),
 ('Dobrý Will Hunting', 'Čtyři pokoje', 0.01980794733120228),
 ('Smrt krásných srnců', 'Marťan', 0.04548467110836407),
 ('Star Trek X: Nemesis', 'Mariňák', 0.049249620125091376),
 ('Vykoupení z věznice Shawshank',
  'Tenkrát v Hollywoodu',
  0.020873813277982434),
 ('Dvanác